In [268]:
# intial imports
import ipywidgets as widgets
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt
import asyncio
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual

In [269]:
# code for cleaning data.csv
file_path = Path("data.csv")
data = pd.read_csv(file_path)
year_limit=1960
data_1960_df = data[data['year']>year_limit]
data_1960_df.describe()
cleanData = data_1960_df[['valence', 'year', 'acousticness', 'danceability',
       'duration_ms', 'energy', 'explicit', 'instrumentalness', 'key',
       'liveness', 'loudness', 'mode', 
       'speechiness', 'tempo', 'popularity']]



In [270]:
# Read csv file
csv_path = Path('play_count_v2.csv')
playcount_df=pd.read_csv(csv_path)
playcount_df.head()

,name,popularity,play_count,revenue_for_plays
0,Dakiti,100,500002027,2218508.99
1,Mood (feat. iann dior),99,741554343,3290276.62
2,Dynamite,97,624093677,2769103.64
3,Blinding Lights,96,1874761848,8318318.32
4,WAP (feat. Megan Thee Stallion),96,640449618,2841674.96


In [271]:
# Group songs by popularity level and get the average play count for each level
playcount_grp_df = playcount_df.groupby("popularity").mean()

In [272]:
# Convert play count to an integer
playcount_grp_df["play_count"] = playcount_grp_df["play_count"].astype(int)

In [273]:
# Convert revenue to an integer
playcount_grp_df["revenue_for_plays"] = playcount_grp_df["revenue_for_plays"].astype(int)

In [274]:
# Diplay first five rows of the dataframe
playcount_grp_df.head()

,play_count,revenue_for_plays
popularity,,
0,7303,32
1,2854,12
2,4014,17
3,7828,34
4,113971,505


In [275]:
playcount_grp_df.reset_index()

,popularity,play_count,revenue_for_plays
0,0,7303,32
1,1,2854,12
2,2,4014,17
3,3,7828,34
4,4,113971,505
...,...,...,...
95,95,368047758,1633027
96,96,823660292,3654580
97,97,624093677,2769103
98,99,741554343,3290276


In [276]:
# create sliders
style = {'description_width': 'initial',"handle_color" : "blue", "width":"200%"}
layout = {"width":"50%", "height":"200%", "padding":"2%", "grid_column":"True"}
layout1 = {"width":"25%", "height":"200%", "padding":"2%", "grid_column":"True"}
widgets.interact_manual.opts['manual_name'] = 'Calculate Revenue'

Title = widgets.Text("Project Spotify Streaming Revenue", style=style, layout=layout1)
# loudness
loudness_widget = widgets.FloatSlider(
    value=1,
    min=-60,
    max=4.0,
    step=0.01,
    description='Loudness:',
    style=style,
    layout=layout,
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

# acousticness
acousticness_widget = widgets.FloatSlider(
    value=1,
    min=0,
    max=1.0,
    step=0.01,
    description='Acousticness:',
    style=style,
    layout=layout,
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

# danceability
danceability_widget = widgets.FloatSlider(
    value=1,
    min=0,
    max=1.0,
    step=0.01,
    description='Danceability:',
    style=style,
    layout=layout,
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

# duration_ms
duration_ms_widget = widgets.IntSlider(
    value=2500000,
    min=5108,
    max=5403500,
    step=500,
    description='Duration (in milliseconds):',
    style=style,
    layout=layout,
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    #readout_format='.1f',
)

#instrumentalness
instrumentalness_widget = widgets.FloatSlider(
    value=1,
    min=0,
    max=1.0,
    step=0.01,
    description='Instrumentalness:',
    style=style,
    layout=layout,
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

# tempo
tempo_widget = widgets.FloatSlider(
    value=120,
    min=0,
    max=243.507,
    step=0.010,
    description='Tempo:',
    style=style,
    layout=layout,
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

# valence
valence_widget = widgets.FloatSlider(
    value=1,
    min=0,
    max=1.0,
    step=0.01,
    description='Valence:',
    style=style,
    layout=layout,
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)


In [277]:
# load model

from tensorflow.keras.models import model_from_json
filepath = 'model.json'
with open(filepath, 'r') as json_file:
  model_json = json_file.read()

loaded_model = model_from_json(model_json)

filepath = 'model.h5'
loaded_model.load_weights(filepath)

year=2020
energy=0.454
explicit=1
key=1
liveness=0.0824
mode=1
speechiness=0.375

In [278]:
@widgets.interact_manual(acousticness_widget=acousticness_widget, danceability_widget=danceability_widget, duration_ms_widget=duration_ms_widget, instrumentalness_widget=instrumentalness_widget, tempo_widget=tempo_widget, valence_widget=valence_widget)
def popularity_predictor(Title=Title,loudness_widget=loudness_widget,acousticness_widget=acousticness_widget, danceability_widget=danceability_widget,
    duration_ms_widget=duration_ms_widget, instrumentalness_widget=instrumentalness_widget, tempo_widget = tempo_widget, valence_widget=valence_widget):
    loudness = loudness_widget
    acousticness = acousticness_widget
    danceability = danceability_widget
    duration_ms = duration_ms_widget
    instrumentalness = instrumentalness_widget
    tempo = tempo_widget
    valence = valence_widget
    year=2020
    energy=0.454
    explicit=1
    key=1
    liveness=0.0824
    mode=1
    speechiness=0.375

    input = [[valence, year, acousticness, danceability,
       duration_ms, energy, explicit, instrumentalness, key,
       liveness, loudness, mode, 
       speechiness, tempo]]
    X2 = cleanData.iloc[:, 0:14].values
    X2[[-1]] = input
    scaler = StandardScaler().fit(X2)
    X2 = scaler.transform(X2)
    prediction = loaded_model.predict(X2[[-1]])
    popularity_score=prediction[0][0]
    rounded_popularity_score = round(popularity_score)

    for index,revenue in playcount_grp_df.iterrows():
      if index == rounded_popularity_score:
        money = revenue["revenue_for_plays"]

    formatted = "${:,.2f}".format(money)



    return formatted


interactive(children=(Text(value='Project Spotify Streaming Revenue', description='Title', layout=Layout(grid_…